<br><br>

## **Import necessary Python libraries and modules**

First, we will import necessary Python libraries and modules. These include as `gdown`, for downloading large files from Google Drive (where we will get our UCSD Goodreads reviews), as well as scikit-learn (`sklearn`) and PyTorch (`torch`), for various machine learning tools.

In [1]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch



To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.5 MB/s 
     |████████████████████████████████| 67 kB 2.6 MB/s 
     |████████████████████████████████| 6.8 MB 28.8 MB/s 
     |████████████████████████████████| 895 kB 35.1 MB/s 
     |████████████████████████████████| 596 kB 39.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Once `transformers` is installed, we will import modules for `DistilBert`, a *distilled* or smaller version of a BERT model that runs more quickly and uses less computing power. This makes it ideal for those just getting started with BERT.

In [2]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [3]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'distilbert-base-multilingual-cased'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 512                                                        

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'Emotion_23'  

In [5]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [6]:
train=pd.read_csv("train23_emo.csv")
train

,Unnamed: 0,Sentence,Emotions
0,0,i didnt want to stay in this feeling of loneli...,Sadness
1,1,Thanks for your reply! Maybe I’ll try that gam...,Joy
2,2,"It's hard to not love guys like [NAME], [NAME]...",Neutral
3,3,ive got a cough that is deep in my chest and o...,Sadness
4,4,"How dare you, she's in your flair too!",Anger
...,...,...,...
32140,32140,멍하니 생각이 잠기게 되는 영화였다..댓글로 알게된 사실이지만 구혜선이 감독이였다니...,Fear
32141,32141,tv 전기세가 아깝다!!!,Surprise
32142,32142,아하하하.. 보다 죽는다.. 내말 믿어라..,Surprise
32143,32143,반담의 팬이라면 꼭 봐야할 영화중 하나!,Anger


In [4]:
#test=pd.read_csv("test2_emo.csv")
#test=pd.read_csv("test_new.csv")
test=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_Test_emotion\\Strategy_2\\test2_emo.csv")

test

,Unnamed: 0,Sentence,Emotions
0,0,Sadly this story has died down why is he bring...,Sadness
1,1,“That’s not my [NAME]” [NAME],Neutral
2,2,I actually live in a place called Star City (i...,Surprise
3,3,How cold is it? I keep hearing about Chicago t...,Neutral
4,4,i write on my blog here that i want or i am go...,Fear
...,...,...,...
5761,5761,넘 잼나거..여자들만의 이야기라 더 관심이 간다..,Disgust
5762,5762,문채원이. 몽골말을 한국말로 함. 억양도 공부 안하고 찍은듯. 보다가 짜증났던 기억임.,Surprise
5763,5763,해석에 의존하는 전형적인 예술장르 영화.,Fear
5764,5764,은근 재미있고 은은한 매력이있는 요런풍의 영화가많았으면 좋겠다.,Joy


In [8]:
#creiamo le liste di train e test. 1.8k train 250 test  
train_texts=train['Sentence']
train_labels=train['Emotions']
test_texts=test['Sentence']
test_labels=test['Emotions']


<br><br>

## **Split the data into training and test sets**

In [9]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(32145, 32145, 5766, 5766)

Here's an example of a training label and review:

In [10]:
train_labels[0], train_texts[0]

('Sadness',
 'i didnt want to stay in this feeling of loneliness the emptiness of my prayers blank requests to a paper deity')

<br><br>

## **Encode data for BERT**

We're going to transform our texts and labels into a format that BERT (via Huggingface and PyTorch) will understand. This is called *encoding* the data.

Here are the steps we need to follow:

1. The labels&mdash;in this case, Goodreads genres&mdash;need to be turned into integers rather than strings.

2. The texts&mdash;in this case, Goodreads reviews&mdash;need to be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens. The tokens, or words in the texts, also need to be separated into "word pieces" and matched to their embedding vectors.

3. We need to add special tokens to help BERT:

| BERT special token | Explanation |
| --------------| ---------|
| [CLS] | Start token of every document. |
| [SEP] | Separator between each sentence |
| [PAD] | Padding at the end of the document as many times as necessary, up to 512 tokens |
|  &#35;&#35; | Start of a "word piece" |




Here we will load `DistilBertTokenizerFast` from the HuggingFace library, which will do all the work of encoding the texts for us. The `tokenizer()` will break word tokens into word pieces, truncate to 512 tokens, and add padding and special BERT tokens.

In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Here we will create a map of our labels, or Goodreads genres, to integer keys. We take the unique labels, and then we make a dictionary that associates each label/tag with an integer.

**Note:** HuggingFace documentation sometimes refers to "labels" as "tags" but these are the same thing. We use "labels" throughout this notebook for clarity.

In [12]:
unique_labels = set(label for label in train_labels)
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

In [13]:
unique_labels

{'Anger', 'Disgust', 'Fear', 'Joy', 'Neutral', 'Sadness', 'Surprise'}

In [14]:
label2id.keys()

dict_keys(['Disgust', 'Joy', 'Sadness', 'Surprise', 'Neutral', 'Fear', 'Anger'])

In [15]:
id2label.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6])

Now let's encode our texts and labels!

In [16]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]

**Examine a Goodreads review in the training set after encoding**

In [17]:
' '.join(train_encodings[29].tokens[0:100])

'[CLS] Is there a university in China ? Are we in 501 ##9 ? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

**Examine a Goodreads review in the test set after encoding**

In [18]:
' '.join(test_encodings[0].tokens[0:100])

'[CLS] Sad ##ly this story has died down why is he bringing it back up . Just let it die . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

**Examine the training labels after encoding**

In [19]:
set(train_labels_encoded)

{0, 1, 2, 3, 4, 5, 6}

**Examine the test labels after encoding**

In [20]:
set(test_labels_encoded)

{0, 1, 2, 3, 4, 5, 6}

<br><br>

## **Make a custom Torch dataset**

Here we combine the encoded labels and texts into dataset objects. We use the custom Torch `MyDataSet` class to make a `train_dataset` object from  the `train_encodings` and `train_labels_encoded`. We also make a `test_dataset` object from `test_encodings`, and `test_labels_encoded`.

In [21]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [22]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [23]:
' '.join(train_dataset.encodings[0].tokens[0:100])

'[CLS] i didn ##t want to stay in this feeling of lo ##nel ##iness the em ##pti ##ness of my prayer ##s bl ##ank request ##s to a paper dei ##ty [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [24]:
' '.join(test_dataset.encodings[1].tokens[0:100])

'[CLS] [UNK] That [UNK] s not my [ NA ##ME ] [UNK] [ NA ##ME ] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

<br><br>

## **Load pre-trained BERT model**

Here we load a pre-trained DistilBERT model and send it to CUDA.

**Note:** If you decide to repeat fine-tuning after already running the following cells, make sure that you re-run this cell to re-load the original pre-trained model before fine-tuning again.

In [25]:
# The model_name needs to match the name used for the tokenizer above.
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classif

<br><br>

## **Set the BERT fine-tuning parameters**

These are the arguments we'll set in the HuggingFace TrainingArguments objects, which we'll then pass to the HuggingFace Trainer object. There are many more possible arguments, but here we highlight the basics and some common gotchas.

When training your own model, you should search over these parameters to find the best settings for your particular dataset. You should use a held-out set of validation data for this step.

| Parameter | Explanation |
|-----------| ------------|
| num_train_epochs | total number of training epochs (how many times to pass through the entire dataset; too much can cause overfitting) |
| per_device_train_batch_size | batch size per device during training |
| per_device_eval_batch_size |  batch size for evaluation |
|  warmup_steps |  number of warmup steps for learning rate scheduler (set lower because of small dataset size) |
| weight_decay | strength of weight decay (reduces size of weights, like regularization) |
| output_dir | output directory for the fine-tuned model and configuration files |
| logging_dir | directory for storing logs |
| logging_steps | how often to print logging output (so that we can stop training early if the loss isn't going down) |
| evaluation_strategy | evaluate while training so that we can see the accuracy going up |

In [27]:
training_args=TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
)

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [28]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

**Note:** This is what we've been aiming for this whole time! All the work of tokenizing, creating datasets, and setting the training arguments was for this cell.

In [29]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function 
)

Time to finally fine-tune! 

Be patient; if you've set everything in Colab to use GPUs, then it should only take a minute or two to run, but if you're running on CPU, it can take hours.

After every 10 steps (as we specified in the TrainingArguments object), the trainer will output the current state of the model, including the training loss, validation ("test") loss, and accuracy (from our `compute_metrics` function).

You should see the loss going down and the accuracy going up. If instead they are staying the same or oscillating, you probably need to change the fine-tuning parameters.

In [30]:
trainer.train()            

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 32145
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Step,Training Loss,Validation Loss,Accuracy
500,1.684200,1.576502,0.366285
1000,1.497300,1.400299,0.465140
1500,1.395100,1.303056,0.504509
2000,1.305600,1.219465,0.542664
2500,1.170200,1.178729,0.552029
3000,1.133000,1.124042,0.578391
3500,1.118500,1.093113,0.590010
4000,1.104500,1.084026,0.596080
4500,0.958900,1.066457,0.608914
5000,0.939700,1.059210,0.611863


***** Running Evaluation *****
  Num examples = 5766
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5766
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5766
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5766
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model

TrainOutput(global_step=8040, training_loss=1.085186479815203, metrics={'train_runtime': 11227.6401, 'train_samples_per_second': 11.452, 'train_steps_per_second': 0.716, 'total_flos': 1.01473124025402e+16, 'train_loss': 1.085186479815203, 'epoch': 4.0})

<br><br>

## **Save fine-tuned model**

The following cell will save the model and its configuration files to a directory in Colab. To preserve this model for future use, you should download the model to your computer.

In [31]:
trainer.save_model(cached_model_directory_name)

Saving model checkpoint to Emotion_23
Configuration saved in Emotion_23/config.json
Model weights saved in Emotion_23/pytorch_model.bin


(Optional) If you've already fine-tuned and saved the model, you can reload it using the following line. You don't have to run fine-tuning every time you want to evaluate.

<br><br>

## **Evaluate fine-tuned model**

The following function of the `Trainer` object will run the built-in evaluation, including our `compute_metrics` function.

In [32]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5766
  Batch size = 16


{'epoch': 4.0,
 'eval_accuracy': 0.6411723898716615,
 'eval_loss': 1.0134658813476562,
 'eval_runtime': 142.3526,
 'eval_samples_per_second': 40.505,
 'eval_steps_per_second': 2.536}

But we might want to do more fine-grained analysis of the model, so we extract the predicted labels.

In [33]:
predicted_results = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 5766
  Batch size = 16


In [34]:
predicted_results.predictions.shape

(5766, 7)

In [35]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [36]:
len(predicted_labels)

5766

In [37]:
print(classification_report(test_labels, 
                            predicted_labels))

              precision    recall  f1-score   support

       Anger       0.64      0.61      0.63       914
     Disgust       0.49      0.50      0.50       506
        Fear       0.58      0.61      0.59       544
         Joy       0.66      0.69      0.68      1099
     Neutral       0.74      0.73      0.74      1048
     Sadness       0.70      0.66      0.68       938
    Surprise       0.55      0.56      0.56       717

    accuracy                           0.64      5766
   macro avg       0.62      0.62      0.62      5766
weighted avg       0.64      0.64      0.64      5766



In [ ]:
#buone performance, tuttavia questo modello perde 5 punti di accuracy rispetto al modello con più KO rev 

In [ ]:
#RELOAD THE MODEL

In [5]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
#tok = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.
tok = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.
mod = DistilBertForSequenceClassification.from_pretrained(cached_model_directory_name)

In [ ]:
#valutazione per domain

In [10]:
def prediction_test(dataset,modello):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
    test_texts=dataset['Sentence']
    test_labels=dataset['Emotions']

    #unique_labels = set(label for label in test_labels)
    #label2id = {label: id for id, label in enumerate(unique_labels)}
    #id2label = {id: label for label, id in label2id.items()}
    unique_labels={'Disgust','Joy','Sadness','Surprise','Neutral','Fear','Anger'}
    label2id={'Disgust':0,'Joy':1,'Sadness':2,'Surprise':3,'Neutral':4,'Fear':5,'Anger':6}
    id2label={0:'Disgust',1:'Joy',2:'Sadness',3:'Surprise',4:'Neutral',5:'Fear',6:'Anger'}
    


#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=max_length)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    
    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

#len(predicted_labels)

    return print(classification_report(test_labels,predicted_labels))#,predicted_labels

In [11]:
prediction_test(test[:2500],mod)   

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at C:\Users\Fossati/.cache\huggingface\transformers\28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\4708

              precision    recall  f1-score   support

       Anger       0.70      0.70      0.70       413
     Disgust       0.58      0.39      0.47        28
        Fear       0.83      0.80      0.81       138
         Joy       0.80      0.78      0.79       506
     Neutral       0.73      0.72      0.73       776
     Sadness       0.83      0.88      0.85       401
    Surprise       0.62      0.67      0.64       238

    accuracy                           0.75      2500
   macro avg       0.73      0.71      0.71      2500
weighted avg       0.75      0.75      0.75      2500



In [12]:
prediction_test(test[2500:3266],mod)   

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at C:\Users\Fossati/.cache\huggingface\transformers\28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\4708

              precision    recall  f1-score   support

       Anger       0.62      0.61      0.62        54
     Disgust       0.60      0.60      0.60        48
        Fear       0.69      0.56      0.62        16
         Joy       0.73      0.66      0.69       151
     Neutral       0.78      0.75      0.76       272
     Sadness       0.67      0.80      0.73        83
    Surprise       0.67      0.73      0.70       142

    accuracy                           0.71       766
   macro avg       0.68      0.67      0.67       766
weighted avg       0.71      0.71      0.71       766



In [13]:
prediction_test(test[3266:],mod)   


loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at C:\Users\Fossati/.cache\huggingface\transformers\28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\4708

              precision    recall  f1-score   support

       Anger       0.59      0.53      0.56       447
     Disgust       0.48      0.49      0.49       430
        Fear       0.50      0.54      0.52       390
         Joy       0.51      0.61      0.55       442
     Neutral       0.00      0.00      0.00         0
     Sadness       0.55      0.45      0.50       454
    Surprise       0.45      0.42      0.43       337

    accuracy                           0.51      2500
   macro avg       0.44      0.44      0.43      2500
weighted avg       0.51      0.51      0.51      2500



C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#modello paragonabile a 22, ovviasmnete dimnuendo le EN e le KO, i risultati di IT si alzano, abbassandosi leggermenti quelli EN e KO
#a livello generale perde solo 1 punto da 22

In [15]:
# tuttavia non siamo soddisfatti dai risultati per il coreano
#forse potrebbe essere cosa buona il fatto di togliere qualche altra emozione, rimaneggiare i dataset per vedere se il modello fatica di meno

In [16]:
#ad ogni modo, la stretgia vincete tra 1 e 2 è la 2, con il set di dati inglese social, piuttosto che paragrafi narrativa

In [ ]:
#cerchiamo di migliorare l'analisi ripartendo da questo modello 23, che risulta essere vincente

In [ ]:
#evidenza importante generale, riscontrata sia per polarity che per emotion: il bilanciamento a livello di lingua è cosa buona e giusta

In [ ]:
#ci si può permettere anche di sovrestare un po'IT dataset, dal momento che comunque prende qualcosa da EN. Non è totalmente differente
#come lingua, a differenza di KO. KO ha bisogno di un pò più di dati essendo diverso a livello sintattico anche